In [4]:
import geopandas as gpd
import duckdb
import os
from tqdm import tqdm

local_tile_dir = "/home/christopher.x.ren/embeddings/ra_tea/tiles"
valid_tile_dir = "/home/christopher.x.ren/embeddings/ra_tea/valid_tiles"

# Create valid_tiles directory if it doesn't exist
os.makedirs(valid_tile_dir, exist_ok=True)

# Get all parquet files
parquet_files = [os.path.join(local_tile_dir, f) for f in os.listdir(local_tile_dir) if f.endswith('.parquet')]

# Read and resave each parquet file
for parquet_file in tqdm(parquet_files):
    gdf = gpd.read_parquet(parquet_file)
    output_file = os.path.join(valid_tile_dir, os.path.basename(parquet_file))
    gdf.to_parquet(output_file)

100%|██████████| 63/63 [00:31<00:00,  2.02it/s]


## Create duckdb table of tiles, index on geometry and tile_id

In [8]:
# Create DuckDB connection
con = duckdb.connect()

# Register spatial extension
con.execute("INSTALL spatial;")
con.execute("LOAD spatial;")

# Create table from parquet files
con.execute(f"""
CREATE TABLE tiles AS 
SELECT * FROM parquet_scan('{valid_tile_dir}/*.parquet')
""")

# Create indexes
con.execute("CREATE INDEX tile_id_idx ON tiles(tile_id)")
con.execute("CREATE INDEX geometry_idx ON tiles(geometry)")

def get_closest_tile(lat: float, lon: float) -> str:
    # Create point from lat/lon
    query = f"""
    WITH point AS (
        SELECT ST_Point({lon}, {lat}) as geom
    )
    SELECT tile_id, ST_Distance(geometry, point.geom) as dist 
    FROM tiles, point
    ORDER BY dist ASC
    LIMIT 1;
    """
    result = con.execute(query).fetchone()
    return result[0]

# Example usage
test_lat, test_lon = 37.7749, -122.4194  # San Francisco coordinates
closest_tile = get_closest_tile(test_lat, test_lon)
print(f"Closest tile to ({test_lat}, {test_lon}): {closest_tile}")


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Closest tile to (37.7749, -122.4194): 47NLC_32_16_10_44_0


## try joining with embeddings database

In [16]:
# Connect to embeddings database
embeddings_db = "/home/christopher.x.ren/embeddings/ra_tea/embeddings.db"
embeddings_con = duckdb.connect(embeddings_db)
embeddings_con.execute("INSTALL spatial;")
embeddings_con.execute("LOAD spatial;")
# Attach embeddings database to main connection
#con.execute(f"ATTACH '{embeddings_db}' AS embeddings")

# Join tiles with embeddings
con.execute("""
    CREATE TABLE tiles_with_embeddings AS
    SELECT t.*, e.*
    FROM tiles t
    LEFT JOIN embeddings.embeddings e ON t.tile_id = e.tile_id
""")


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

BinderException: Binder Error: Referenced column "embedding" not found in FROM clause!
Candidate bindings: "point.geom", "tiles_with_embeddings.geometry"
LINE 5:     SELECT tile_id, embedding, ST_Distance(geometry, point.geom) as dist
    FROM tiles_with_embeddings, point 
    ORDER BY dist ASC
    LIMIT 1;
    ...
                            ^

In [18]:

def get_closest_tile_embedding(lat: float, lon: float):
    # Create point from lat/lon
    query = f"""
    WITH point AS (
        SELECT ST_Point({lon}, {lat}) as geom
    )
    SELECT *
    FROM tiles_with_embeddings, point 
    ORDER BY ST_Distance(geometry, point.geom) ASC
    LIMIT 1;
    """
    result = con.execute(query).fetchone()
    return result

# Example usage 
test_lat, test_lon = 37.7749, -122.4194  # San Francisco coordinates
embedding = get_closest_tile_embedding(test_lat, test_lon)
print(f"Closest tile to ({test_lat}, {test_lon}): {closest_tile_id}")
print(f"Embedding shape: {len(embedding)}")


NameError: name 'closest_tile_id' is not defined

In [22]:
embedding[0]

b'\x02\x04\x00\x00\x00\x00\x00\x00Rg\x92HL\x83QHS\x8f\x92HM\xd3QH\x02\x00\x00\x00\x01\x00\x00\x00\x05\x00\x00\x00\x00\x00\x00\x006\xfaAO\xeaQ\x12A\x8a\x95h\x81i:\nA6\xfaAO\xeaQ\x12A\x8a\x95h\x81i0\nA6\xfaAO\xeaL\x12A\x8a\x95h\x81i0\nA6\xfaAO\xeaL\x12A\x8a\x95h\x81i:\nA6\xfaAO\xeaQ\x12A\x8a\x95h\x81i:\nA'

In [15]:
# Get table names
print("Tables:")
print(embeddings_con.execute("SHOW TABLES").fetchall())

# Get column names for each table
print("\nColumns:")
for table in embeddings_con.execute("SHOW TABLES").fetchall():
    table_name = table[0]
    print(f"\n{table_name}:")
    print(embeddings_con.execute(f"DESCRIBE {table_name}").fetchall())

Tables:
[('embeddings',)]

Columns:

embeddings:
[('vit-dino-patch16_0', 'DOUBLE', 'YES', None, None, None), ('vit-dino-patch16_1', 'DOUBLE', 'YES', None, None, None), ('vit-dino-patch16_2', 'DOUBLE', 'YES', None, None, None), ('vit-dino-patch16_3', 'DOUBLE', 'YES', None, None, None), ('vit-dino-patch16_4', 'DOUBLE', 'YES', None, None, None), ('vit-dino-patch16_5', 'DOUBLE', 'YES', None, None, None), ('vit-dino-patch16_6', 'DOUBLE', 'YES', None, None, None), ('vit-dino-patch16_7', 'DOUBLE', 'YES', None, None, None), ('vit-dino-patch16_8', 'DOUBLE', 'YES', None, None, None), ('vit-dino-patch16_9', 'DOUBLE', 'YES', None, None, None), ('vit-dino-patch16_10', 'DOUBLE', 'YES', None, None, None), ('vit-dino-patch16_11', 'DOUBLE', 'YES', None, None, None), ('vit-dino-patch16_12', 'DOUBLE', 'YES', None, None, None), ('vit-dino-patch16_13', 'DOUBLE', 'YES', None, None, None), ('vit-dino-patch16_14', 'DOUBLE', 'YES', None, None, None), ('vit-dino-patch16_15', 'DOUBLE', 'YES', None, None, None), 